# 🎵 Complete Audio Restoration Pipeline with Demucs

## ✨ Features:

**Full Professional Audio Processing:**
- ✅ **Spectral Analysis** - Analyze audio quality and issues
- ✅ **Noise Reduction** - Remove background noise intelligently
- ✅ **Frequency Restoration** - Restore missing high frequencies
- ✅ **Stem Separation with Demucs** - Separate vocals, drums, bass, other (state-of-the-art AI)
- ✅ **Mastering** - Professional EQ, compression, limiting
- ✅ **LUFS Normalization** - Streaming-ready loudness

## 🔧 Technical Details:

**Demucs Integration:**
- Uses `htdemucs` model (4 stems: vocals, drums, bass, other)
- Automatic FFmpeg installation (required dependency)
- Automatic TorchCodec installation (required for file saving)
- GPU acceleration if available
- Fallback to basic HPSS method if Demucs fails

## ⏱️ Processing Time:
- **Without Demucs:** 1-2 minutes per track
- **With Demucs (CPU):** 10-20 minutes per track
- **With Demucs (GPU):** 3-5 minutes per track

## 🚀 Quick Start:

1. Run all cells in order
2. Upload your audio file
3. Choose a processing configuration
4. Get professional results!

---

## 📦 Step 1: Install Dependencies

**This will install:**
- Audio processing libraries (librosa, soundfile, scipy)
- Demucs for stem separation
- TorchCodec for Demucs file operations
- FFmpeg (system package)

**Time:** ~2-3 minutes

In [ ]:
%%capture
# Install Python packages
!pip install -q librosa soundfile scipy numpy matplotlib ipywidgets
!pip install -q demucs torchcodec

print("✓ Python packages installed!")

In [ ]:
# Install FFmpeg (required for Demucs)
print("🔍 Checking FFmpeg...")
import subprocess

ffmpeg_check = subprocess.run(['ffmpeg', '-version'], capture_output=True)

if ffmpeg_check.returncode != 0:
    print("⚠️  FFmpeg not found. Installing...")
    !apt-get update -qq
    !apt-get install -y -qq ffmpeg
    print("✓ FFmpeg installed!")
else:
    print("✓ FFmpeg already installed!")

# Verify installation
!ffmpeg -version | head -1

## 📥 Step 2: Download Audio Restoration Pipeline

**Downloads the complete pipeline from GitHub**

**Time:** ~5 seconds

In [ ]:
import os
import sys

# Remove old version if exists
!rm -rf /content/audio-pipeline

# Clone the repository
print("📥 Downloading audio restoration pipeline...")
!git clone -q -b claude/audio-restoration-pipeline-gAFxk \
  https://github.com/guitorte/musicas.git \
  /content/audio-pipeline

# Add to Python path
sys.path.insert(0, '/content/audio-pipeline/audio-restoration-pipeline')

print("✓ Pipeline downloaded!")
print("✓ Ready to process audio!")

## 📂 Step 3: Mount Google Drive & Select File

**Mounts your Google Drive and lets you choose which file to restore from `00-restore` folder**

In [ ]:
from google.colab import drive
import os
import glob
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

# Mount Google Drive
print("📂 Mounting Google Drive...")
drive.mount('/content/drive')

# Define input/output directories
AUDIO_INPUT_DIR = '/content/drive/MyDrive/00-restore'
AUDIO_OUTPUT_DIR = '/content/drive/MyDrive/00-restore/restored_output'

print(f"✓ Drive mounted!")
print(f"📁 Input folder: {AUDIO_INPUT_DIR}")
print(f"📁 Output folder: {AUDIO_OUTPUT_DIR}")

# Create output directory
os.makedirs(AUDIO_OUTPUT_DIR, exist_ok=True)

print("\n" + "="*70)

# Find all audio files
audio_files = []
for ext in ['*.mp3', '*.wav', '*.MP3', '*.WAV', '*.m4a', '*.M4A', '*.flac', '*.FLAC']:
    audio_files.extend(glob.glob(os.path.join(AUDIO_INPUT_DIR, ext)))

audio_files = sorted(set(audio_files))

print(f"✓ Found {len(audio_files)} audio file(s)\n")

if not audio_files:
    print(f"⚠️  No audio files found in {AUDIO_INPUT_DIR}")
    print("   Please add audio files to your '00-restore' folder in Google Drive")
else:
    # List all files
    for i, file in enumerate(audio_files, 1):
        size_mb = os.path.getsize(file) / (1024 * 1024)
        name = Path(file).name
        print(f"{i:2d}. {name[:55]:55s} ({size_mb:6.2f} MB)")
    
    print("\n" + "="*70)
    
    # Create dropdown options with file info
    file_options = []
    for i, file in enumerate(audio_files):
        size_mb = os.path.getsize(file) / (1024 * 1024)
        name = Path(file).name
        file_options.append((f"{i+1}. {name} ({size_mb:.2f} MB)", i))
    
    # Create dropdown widget
    file_selector = widgets.Dropdown(
        options=file_options,
        value=0,
        description='Select file:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='90%')
    )
    
    # Info label for selected file
    info_label = widgets.HTML(value="")
    
    # Function to update info when selection changes
    def on_file_change(change):
        idx = change['new']
        selected_file = audio_files[idx]
        size_mb = os.path.getsize(selected_file) / (1024 * 1024)
        duration_info = ""
        try:
            import librosa
            total_duration = librosa.get_duration(path=selected_file)
            duration_info = f"<br><b>Duration:</b> {total_duration:.1f}s ({total_duration/60:.1f} min)"
        except:
            pass
        info_label.value = f"<b>Selected:</b> {Path(selected_file).name}<br><b>Size:</b> {size_mb:.2f} MB{duration_info}"
    
    # Connect event
    file_selector.observe(on_file_change, names='value')
    
    # Trigger initial display
    on_file_change({'new': 0})
    
    print("\n🎵 SELECT THE FILE TO RESTORE:")
    display(file_selector)
    display(info_label)
    
    # Set global variables for use in other cells
    audio_file = Path(audio_files[0]).name
    audio_path = audio_files[0]
    
    def get_selected_file():
        """Returns the currently selected file path"""
        return audio_files[file_selector.value]
    
    def get_selected_filename():
        """Returns just the filename of the selected file"""
        return Path(audio_files[file_selector.value]).name
    
    print("\n💡 Use the dropdown above to select your file, then run the next cells")

## 📊 Step 4: Analyze Audio

**Analyzes your audio to determine optimal processing settings**

In [ ]:
import librosa
import numpy as np
from pathlib import Path

# Use selected file from dropdown
if 'file_selector' in dir():
    audio_path = get_selected_file()
else:
    print("⚠️  Run Step 3 first to select a file!")
    audio_path = None

if audio_path:
    print("="*70)
    print(f"🎵 ANALYZING: {Path(audio_path).name}")
    print("="*70)

    # Load audio
    y, sr = librosa.load(audio_path, sr=44100)
    duration = len(y) / sr

    # Basic metrics
    print(f"\n📊 Basic Info:")
    print(f"  Duration: {duration:.1f}s ({duration/60:.1f} minutes)")
    print(f"  Sample Rate: {sr} Hz")
    print(f"  Samples: {len(y):,}")

    # Loudness analysis
    rms = np.sqrt(np.mean(y**2))
    lufs_approx = 20 * np.log10(rms + 1e-10) + 0.691
    peak = np.max(np.abs(y))
    dynamic_range = 20 * np.log10(peak / (rms + 1e-10))

    print(f"\n🔊 Loudness:")
    print(f"  LUFS: {lufs_approx:.1f} dB")
    print(f"  Peak: {peak:.3f} ({20*np.log10(peak):.1f} dBFS)")
    print(f"  Dynamic Range: {dynamic_range:.1f} dB")

    # Frequency analysis
    S = np.abs(librosa.stft(y))
    freqs = librosa.fft_frequencies(sr=sr)

    bass_mask = (freqs >= 20) & (freqs < 250)
    mid_mask = (freqs >= 250) & (freqs < 2000)
    high_mask = (freqs >= 2000) & (freqs < 8000)
    air_mask = (freqs >= 8000)

    bass_energy = np.mean(S[bass_mask, :])
    mid_energy = np.mean(S[mid_mask, :])
    high_energy = np.mean(S[high_mask, :])
    air_energy = np.mean(S[air_mask, :])

    total_energy = bass_energy + mid_energy + high_energy + air_energy

    print(f"\n🎼 Frequency Balance:")
    print(f"  Bass (20-250 Hz):   {100*bass_energy/total_energy:.1f}%")
    print(f"  Mids (250-2k Hz):   {100*mid_energy/total_energy:.1f}%")
    print(f"  Highs (2-8 kHz):    {100*high_energy/total_energy:.1f}%")
    print(f"  Air (8+ kHz):       {100*air_energy/total_energy:.1f}%")

    # Noise estimation
    noise_floor = np.percentile(np.abs(y), 5)
    signal_peak = np.percentile(np.abs(y), 95)
    snr_estimate = 20 * np.log10(signal_peak / (noise_floor + 1e-10))

    print(f"\n📉 Noise Level:")
    print(f"  SNR: {snr_estimate:.1f} dB")
    if snr_estimate < 20:
        print(f"  ⚠️  High noise - strong reduction recommended")
    elif snr_estimate < 30:
        print(f"  💡 Moderate noise - medium reduction recommended")
    else:
        print(f"  ✓ Clean signal - light reduction only")

    # Quality checks
    print(f"\n✓ Quality Checks:")
    if peak > 0.99:
        print(f"  ⚠️  Clipping detected!")
    else:
        print(f"  ✓ No clipping")

    if lufs_approx < -20:
        print(f"  ⚠️  Very quiet (needs gain)")
    elif lufs_approx > -10:
        print(f"  ⚠️  Very loud (may be over-compressed)")
    else:
        print(f"  ✓ Good loudness")

    print("\n" + "="*70)

## ⚙️ Step 5: Configure Processing

**Choose your processing preset:**

### Available Presets:

1. **`CONFIG_FULL_WITH_DEMUCS`** - Complete restoration with AI stem separation
   - Stem separation with Demucs (vocals, drums, bass, other)
   - Noise reduction, frequency restoration
   - Professional mastering
   - **⏱️ Time:** 10-20 min (CPU) or 3-5 min (GPU)
   - **Best for:** Tracks that need stem-level processing

2. **`CONFIG_STANDARD`** - Professional restoration without Demucs
   - Noise reduction, frequency restoration
   - Professional EQ and mastering
   - **⏱️ Time:** 1-2 minutes
   - **Best for:** Most tracks (recommended)

3. **`CONFIG_CONSERVATIVE`** - Gentle improvements only
   - Light processing, preserves character
   - **⏱️ Time:** 1 minute
   - **Best for:** Already good quality tracks

4. **`CONFIG_MASTERING_ONLY`** - Mastering only (no restoration)
   - EQ, compression, limiting, LUFS normalization
   - **⏱️ Time:** 30 seconds
   - **Best for:** Clean tracks that just need final polish

In [ ]:
# ═══════════════════════════════════════════════════════════════
# CONFIGURATION PRESETS
# ═══════════════════════════════════════════════════════════════

# 1. FULL PROCESSING WITH DEMUCS (Slowest, highest quality)
CONFIG_FULL_WITH_DEMUCS = {
    'remove_clicks': True,
    'reduce_noise': True,
    'noise_reduction_strength': 0.5,
    'restore_frequencies': True,
    'freq_restoration_method': 'harmonic_synthesis',
    'psychoacoustic_enhancement': True,
    'separate_stems': True,  # ⚠️ This enables Demucs (slow!)
    'stem_model': 'demucs',
    'target_lufs': -14.0,
    'master_eq': {
        'bass': 0.5,
        'mid': 0.0,
        'presence': 1.0,
        'treble': 0.8
    },
    'add_presence': True
}

# 2. STANDARD PROFESSIONAL (Fast, no Demucs)
CONFIG_STANDARD = {
    'remove_clicks': True,
    'reduce_noise': True,
    'noise_reduction_strength': 0.4,
    'restore_frequencies': True,
    'freq_restoration_method': 'harmonic_synthesis',
    'psychoacoustic_enhancement': True,
    'separate_stems': False,  # No Demucs - fast processing
    'target_lufs': -14.0,
    'master_eq': {
        'bass': 0.5,
        'mid': 0.0,
        'presence': 1.0,
        'treble': 0.8
    },
    'add_presence': True
}

# 3. CONSERVATIVE (Very fast, gentle)
CONFIG_CONSERVATIVE = {
    'remove_clicks': True,
    'reduce_noise': True,
    'noise_reduction_strength': 0.3,
    'restore_frequencies': False,
    'separate_stems': False,
    'target_lufs': -14.0,
    'master_eq': {
        'bass': 0.0,
        'mid': 0.0,
        'presence': 0.5,
        'treble': 0.3
    },
    'add_presence': False
}

# 4. MASTERING ONLY (Fastest)
CONFIG_MASTERING_ONLY = {
    'remove_clicks': False,
    'reduce_noise': False,
    'restore_frequencies': False,
    'separate_stems': False,
    'target_lufs': -14.0,
    'master_eq': {
        'bass': 0.5,
        'mid': 0.0,
        'presence': 1.0,
        'treble': 0.8
    },
    'add_presence': True
}

# ═══════════════════════════════════════════════════════════════
# SELECT YOUR CONFIG HERE:
# ═══════════════════════════════════════════════════════════════

CONFIG = CONFIG_FULL_WITH_DEMUCS  # ⭐ Change this to select a preset

# Detect which config was selected
config_name = "CUSTOM"
if CONFIG == CONFIG_FULL_WITH_DEMUCS:
    config_name = "CONFIG_FULL_WITH_DEMUCS"
elif CONFIG == CONFIG_STANDARD:
    config_name = "CONFIG_STANDARD"
elif CONFIG == CONFIG_CONSERVATIVE:
    config_name = "CONFIG_CONSERVATIVE"
elif CONFIG == CONFIG_MASTERING_ONLY:
    config_name = "CONFIG_MASTERING_ONLY"

# Display selected config
print("="*70)
print("✓ CONFIGURATION LOADED")
print("="*70)
print(f"\nSelected: {config_name}")
print(f"\nSettings:")
print(f"  • Noise Reduction: {CONFIG['noise_reduction_strength']}")
print(f"  • Frequency Restoration: {CONFIG['restore_frequencies']}")
print(f"  • Stem Separation (Demucs): {CONFIG['separate_stems']}")
print(f"  • Target LUFS: {CONFIG['target_lufs']} dB")

if CONFIG['separate_stems']:
    print(f"\n⚠️  Demucs enabled - this will take 10-20 minutes!")
    print(f"   Set CONFIG = CONFIG_STANDARD for faster processing without Demucs")
else:
    print(f"\n✓ Fast processing - no Demucs (1-2 minutes)")
    print(f"   Set CONFIG = CONFIG_FULL_WITH_DEMUCS to enable stem separation")

print("\n" + "="*70)

## 🎵 Step 6: PROCESS!

**This will process your audio with the selected configuration**

### What happens:
1. **Stage 1:** Spectral analysis
2. **Stage 2:** Noise reduction and cleanup
3. **Stage 3:** Frequency restoration
4. **Stage 4:** Stem separation (if enabled)
5. **Stage 5:** Mastering (EQ, compression, limiting)
6. **Stage 6:** LUFS normalization

### Time estimates:
- **With Demucs:** 10-20 minutes (CPU) or 3-5 minutes (GPU)
- **Without Demucs:** 1-2 minutes

In [ ]:
from modules import AudioRestorationPipeline
from pathlib import Path
import json

# Use selected file from dropdown
if 'file_selector' in dir():
    audio_path = get_selected_file()
else:
    print("⚠️  Run Step 3 first to select a file!")
    audio_path = None

if audio_path:
    print("="*70)
    print("🎵 STARTING AUDIO PROCESSING")
    print("="*70)

    # Initialize pipeline - output to Google Drive
    pipeline = AudioRestorationPipeline(
        sr=44100,
        output_base_dir=AUDIO_OUTPUT_DIR,
        log_dir=os.path.join(AUDIO_OUTPUT_DIR, 'logs')
    )

    # Process
    filename = Path(audio_path).stem
    print(f"\nProcessing: {Path(audio_path).name}")
    print(f"Output folder: {AUDIO_OUTPUT_DIR}")
    print("\n" + "="*70 + "\n")

    result = pipeline.process_audio(
        audio_path,
        output_name=filename,
        config=CONFIG
    )

    print("\n" + "="*70)
    print("✓✓✓ PROCESSING COMPLETE! ✓✓✓")
    print("="*70)

    # Save results
    result_path = os.path.join(AUDIO_OUTPUT_DIR, 'processing_result.json')
    with open(result_path, 'w') as f:
        json.dump(result, f, indent=2)

    final_wav = result['stages']['mastering']['output']
    print(f"\n📁 Final output: {final_wav}")
    print(f"📊 Results: {result_path}")

    # Copy to easy location in Drive
    final_name = f"{filename}_MASTERED.wav"
    final_output_path = os.path.join(AUDIO_OUTPUT_DIR, final_name)
    !cp "{final_wav}" "{final_output_path}"

    print(f"\n✓ Saved to: {final_output_path}")
    print(f"\n💡 File is now in your Google Drive '00-restore/restored_output' folder!")

## 🎧 Step 7: Listen to Results

**Compare before and after**

In [ ]:
from IPython.display import Audio, display
import glob

print("="*70)
print("🎧 BEFORE vs AFTER COMPARISON")
print("="*70)

# Use the selected file
if 'file_selector' in dir():
    audio_path = get_selected_file()
    
    print("\n🔴 BEFORE (Original):")
    display(Audio(audio_path))

    print("\n🟢 AFTER (Mastered):")
    # Find mastered file in Drive output folder
    mastered_files = glob.glob(os.path.join(AUDIO_OUTPUT_DIR, '*_MASTERED.wav'))
    if mastered_files:
        # Use the most recent one
        latest_mastered = max(mastered_files, key=os.path.getctime)
        display(Audio(latest_mastered))
        print(f"   File: {Path(latest_mastered).name}")
    else:
        print("⚠️  Mastered file not found. Run Step 6 first.")

    print("\n💡 Listen with headphones for best results!")
else:
    print("⚠️  Run Step 3 first to select a file!")

## 📊 Step 8: Analysis Report

**See detailed before/after metrics**

In [ ]:
import librosa
import numpy as np

print("="*70)
print("📊 BEFORE vs AFTER ANALYSIS")
print("="*70)

if 'file_selector' in dir():
    audio_path = get_selected_file()
    
    # Load original
    y_orig, sr = librosa.load(audio_path, sr=44100)
    rms_orig = np.sqrt(np.mean(y_orig**2))
    lufs_orig = 20 * np.log10(rms_orig + 1e-10) + 0.691
    peak_orig = np.max(np.abs(y_orig))

    print("\n🔴 ORIGINAL:")
    print(f"  LUFS: {lufs_orig:.1f} dB")
    print(f"  Peak: {peak_orig:.3f}")

    # Load mastered
    mastered_files = glob.glob(os.path.join(AUDIO_OUTPUT_DIR, '*_MASTERED.wav'))
    if mastered_files:
        latest_mastered = max(mastered_files, key=os.path.getctime)
        y_mast, sr = librosa.load(latest_mastered, sr=44100)
        rms_mast = np.sqrt(np.mean(y_mast**2))
        lufs_mast = 20 * np.log10(rms_mast + 1e-10) + 0.691
        peak_mast = np.max(np.abs(y_mast))

        print("\n🟢 MASTERED:")
        print(f"  LUFS: {lufs_mast:.1f} dB")
        print(f"  Peak: {peak_mast:.3f}")

        print("\n📊 IMPROVEMENTS:")
        print(f"  LUFS: {lufs_orig:.1f} → {lufs_mast:.1f} ({lufs_mast - lufs_orig:+.1f} dB)")
        print(f"  Peak: {peak_orig:.3f} → {peak_mast:.3f}")

        print("\n✓ QUALITY CHECKS:")
        if abs(lufs_mast + 14) < 1:
            print("  ✓ Perfect for Spotify/YouTube/Apple Music")
        if peak_mast < 0.99:
            print("  ✓ No clipping detected")
        if abs(lufs_mast - lufs_orig) > 0.5:
            print("  ✓ Loudness normalized successfully")
    else:
        print("\n⚠️  Mastered file not found. Run Step 6 first.")
else:
    print("⚠️  Run Step 3 first to select a file!")

print("\n" + "="*70)

## 📥 Step 9: Download Results

**Download your mastered audio**

In [ ]:
from google.colab import files
import glob
import os

print("📥 DOWNLOAD OPTIONS")
print("="*70)

print("\n✓ Files are already saved to your Google Drive!")
print(f"   Location: {AUDIO_OUTPUT_DIR}")

# List files in output folder
mastered_files = glob.glob(os.path.join(AUDIO_OUTPUT_DIR, '*_MASTERED.wav'))

if mastered_files:
    print(f"\n📁 Mastered Files ({len(mastered_files)}):")
    for wav in mastered_files:
        size = os.path.getsize(wav) / (1024*1024)
        print(f"   • {Path(wav).name} ({size:.1f} MB)")
    
    print("\n💡 To download to your computer, uncomment and run:")
    print("   # files.download(mastered_files[0])")
    
    # Uncomment below to download
    # for wav in mastered_files:
    #     files.download(wav)

# Option 2: Download individual stems (if Demucs was used)
if 'result' in dir() and CONFIG.get('separate_stems', False):
    stem_files = glob.glob(f"{result['output_dir']}/**/*.wav", recursive=True)
    stem_files = [f for f in stem_files if any(stem in f for stem in ['vocals', 'drums', 'bass', 'other'])]
    
    if stem_files:
        print(f"\n📁 Individual Stems ({len(stem_files)}):")
        for stem in stem_files:
            print(f"   • {Path(stem).name}")
        
        print("\n💡 Uncomment below to download stems:")
        # for stem in stem_files:
        #     files.download(stem)

print("\n" + "="*70)
print("✓ All files saved to Google Drive!")

## 🔍 Step 10: View Stems (If Demucs Was Used)

**Listen to individual separated stems**

In [ ]:
if CONFIG.get('separate_stems', False) and 'result' in dir():
    print("="*70)
    print("🎼 INDIVIDUAL STEMS (Demucs Separation)")
    print("="*70)
    
    # Find stems
    stem_files = glob.glob(f"{result['output_dir']}/**/*.wav", recursive=True)
    
    stems = {}
    for f in stem_files:
        name = Path(f).stem.lower()
        if 'vocals' in name:
            stems['vocals'] = f
        elif 'drums' in name:
            stems['drums'] = f
        elif 'bass' in name:
            stems['bass'] = f
        elif 'other' in name:
            stems['other'] = f
    
    # Display each stem
    if 'vocals' in stems:
        print("\n🎤 VOCALS:")
        display(Audio(stems['vocals']))
    
    if 'drums' in stems:
        print("\n🥁 DRUMS:")
        display(Audio(stems['drums']))
    
    if 'bass' in stems:
        print("\n🎸 BASS:")
        display(Audio(stems['bass']))
    
    if 'other' in stems:
        print("\n🎹 OTHER (Instruments):")
        display(Audio(stems['other']))
    
    print("\n" + "="*70)
else:
    print("\n⚠️  Stem separation was not enabled or processing hasn't run yet.")
    print("   Set CONFIG = CONFIG_FULL_WITH_DEMUCS to enable Demucs")
    print("   Then run Step 6 to process the audio")

---

## 💡 Tips & Troubleshooting

### Demucs Issues:

**If Demucs fails with "Could not load libtorchcodec":**
```python
# Run this in a new cell:
!apt-get update && apt-get install -y ffmpeg
# Then re-run the processing cell
```

**If Demucs is too slow:**
- Use `CONFIG_STANDARD` instead of `CONFIG_FULL_WITH_DEMUCS`
- Enable GPU in Colab: Runtime → Change runtime type → GPU

**If memory issues:**
- Process shorter audio segments
- Use `CONFIG_STANDARD` (no Demucs)

### Processing Tips:

- **For most tracks:** Use `CONFIG_STANDARD` (fast, good results)
- **For vocal isolation/remixes:** Use `CONFIG_FULL_WITH_DEMUCS`
- **For already good tracks:** Use `CONFIG_CONSERVATIVE`
- **For final polish only:** Use `CONFIG_MASTERING_ONLY`

### Custom Configuration:

You can create your own config by copying and modifying any preset:

```python
CONFIG_CUSTOM = {
    'reduce_noise': True,
    'noise_reduction_strength': 0.6,  # 0.0-1.0
    'restore_frequencies': True,
    'separate_stems': False,  # Set True to enable Demucs
    'target_lufs': -14.0,  # -11 (loud) to -16 (quiet)
    'master_eq': {
        'bass': 1.0,      # -3.0 to +3.0 dB
        'mid': 0.0,
        'presence': 2.0,
        'treble': 1.5
    }
}

CONFIG = CONFIG_CUSTOM
```

---

## 📚 Documentation

**GitHub Repository:** https://github.com/guitorte/musicas

**Demucs:** State-of-the-art AI music source separation
- Paper: https://arxiv.org/abs/2111.03600
- Separates vocals, drums, bass, and other instruments
- Requires FFmpeg and TorchCodec dependencies

---

**Made with ❤️ for music producers**

🎵 **100% Free • Professional Results • Demucs AI Integration** 🎵

---